In [1]:
# Don't use this or you can't visualise the data for some reason
#import findspark
#import streamlit
#findspark.init()

In [2]:
#Imports
import pyspark
from pyspark import SparkContext, SparkConf
from collections import Counter
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import split, col

from pyspark.sql import SparkSession
#spark = SparkSession.builder.getOrCreate()
spark = SparkSession.builder.master('local[10]').config('spark.driver.memory','10g').getOrCreate()

conf = SparkConf().setAppName('Project').setMaster('local')
sparkContext = SparkContext.getOrCreate(conf=conf) #SparkContext(conf = conf)

/usr/local/lib/python3.12/site-packages/pyspark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/25 10:09:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Read in and assign column names
# Income data
Income = spark.read.csv('income_csv_cleaned_no_commas.csv',header=False)
Income = Income.toDF('Area_code','Area','Self_num_ind','Self_mean','Self_median','Num_ind','Mean','Median','Pens_num_ind','Pens_mean',
                     'Pens_median','Other_num_ind','Other_mean','Other_median','Tot_num_ind','Tot_mean','Tot_median','Tax_num_ind','Tax_mean','Tax_median','Tot_tax')

# Drop not applicable
Income = Income.na.drop()
Income = Income.filter(Income.Area_code!='Not applicable')

# Remove original formatting
Income = Income.filter(Income.Area!='Region/County')

# Population data
Population = spark.read.csv('population_data.csv',header=False)
Population = Population.toDF('Code','Name','Geography','All_ages','0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15',
                     '16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32','33','34','35','36','37',
                     '38','39','40','41','42','43','44','45','46','47','48','49','50','51','52','53','54','55','56','57','58','59',
                     '60','61','62','63','64','65','66','67','68','69','70','71','72','73','74','75','76','77','78','79','80','81','82','83','84','85','86','87','88','89','90+')

# Drop not applicable
Population = Population.na.drop()

# Remove original formatting
Population = Population.filter(Population.Code!='Code')

# Income data
Statements = spark.read.csv('Transposed_Consolidated_Income_Statements.csv',header=False)
Statements = Statements.toDF('Company','Fina_period','State','Revenue','Cost','Gross','Other','Disp_close','Admin','Operating','Fin_out','Fin_in','Share_loss','Before_tax','Tax','Profit')

# Remove original formatting
Statements = Statements.drop(col('Company'))
Statements = Statements.drop(col('Fina_period'))
Statements = Statements.drop(col('State'))

# Remove original formatting
Statements = Statements.filter(Statements.Cost!='Cost of sales')

# Stores data
Stores = spark.read.csv('Stores1.csv',header=False)
Stores = Stores.toDF('Codes','Name','Morrisons','Tesco','Sainsbury')

#Stores.show()

In [4]:
#Population.show()

In [5]:
#Statements.show()

In [6]:
import numpy as np
from pyspark.sql import functions as F
from pyspark.sql.functions import monotonically_increasing_id

# Condensing the data to only what is relevant
# Dropping self employement data

# Income condensed
Cond_Inc = Income

Cond_Inc = Cond_Inc.drop(col('Self_num_ind'))
Cond_Inc = Cond_Inc.drop(col('Self_mean'))
Cond_Inc = Cond_Inc.drop(col('Self_median'))

# Dropping pension data
Cond_Inc = Cond_Inc.drop(col('Pens_num_ind'))
Cond_Inc = Cond_Inc.drop(col('Pens_mean'))
Cond_Inc = Cond_Inc.drop(col('Pens_median'))

# Remove the non-applicable areas
Cond_Inc = Cond_Inc.filter(Cond_Inc.Area_code != 'Not applicable')

# Population condensed
Cond_Pop = Population

# Dropping columns for individual ages
for colu in range(0,90):
    Cond_Pop = Cond_Pop.drop('',col(str(colu)))

Cond_Pop = Cond_Pop.drop(col('90+'))

# Dropping repeated data in both population and income
Cond_Pop = Cond_Pop.drop(col('Name'))

# Join two dataframes
Data = Cond_Pop.join(Cond_Inc,Cond_Inc.Area_code == Cond_Pop.Code,'inner')
Data = Data.drop(col('Area_code'))

# Add new primary key
Data = Data.withColumn("id", monotonically_increasing_id())

# Get all codes in the new dataframe
codes = [row.Code for row in Data.select('Code').collect()]

# Temporary store for string data types with primary key
Temp_Store = Data.select('id','Code','Area','Geography')
Temp_Store = Temp_Store.withColumnRenamed('id', 'id1')

# Drop string data type columns
Data = Data.drop(col('Code'))
Data = Data.drop(col('Area'))
Data = Data.drop(col('Geography'))

# Changes data type
for c in Data.columns:
    Data = Data.withColumn(c, F.col(c).cast(IntegerType()))

# Creates a single database
Cast_Data = Data.join(Temp_Store,Data.id == Temp_Store.id1,'full_outer')
Cast_Data = Cast_Data.drop(col('id1'))

Cast_Data = Cast_Data.filter(Cast_Data.id!=0)

Cast_Data.show()

25/03/25 10:10:06 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------+-------+-----+------+-------------+----------+------------+-----------+--------+----------+-----------+--------+----------+-------+---+---------+--------------------+-------------------+
|All_ages|Num_ind| Mean|Median|Other_num_ind|Other_mean|Other_median|Tot_num_ind|Tot_mean|Tot_median|Tax_num_ind|Tax_mean|Tax_median|Tot_tax| id|     Code|                Area|          Geography|
+--------+-------+-----+------+-------------+----------+------------+-----------+--------+----------+-----------+--------+----------+-------+---+---------+--------------------+-------------------+
| 2711380|    929|30200| 24700|          508|      3840|          31|       1230|   31500|     24900|       1230|    4110|      2230|   5060|  1|E12000001|          North East|             Region|
| 1153212|    396|30500| 25000|          197|      3800|          29|        508|   31700|     25200|        508|    4120|      2290|   2090|  2|E11000007|         Tyne & Wear|Metropolitan County|
| 7600126|   26

In [7]:
# Regions based data
# Create region table
Region = Cond_Pop.join(Stores,Stores.Codes == Cond_Pop.Code,'inner')
Region = Region.drop(col('Codes'))

#Region = Region.filter(Region.Geography!='Country')

Region.show()

+---------+---------+--------+--------------------+---------+-----+---------+
|     Code|Geography|All_ages|                Name|Morrisons|Tesco|Sainsbury|
+---------+---------+--------+--------------------+---------+-----+---------+
|E92000001|  Country|57690323|             ENGLAND|      506| 2529|     1310|
|E12000001|   Region| 2711380|          NORTH EAST|       31| #N/A|     #N/A|
|E12000002|   Region| 7600126|          NORTH WEST|       67| #N/A|     #N/A|
|E12000003|   Region| 5594125|YORKSHIRE AND THE...|       55| #N/A|     #N/A|
|E12000004|   Region| 4991265|       EAST MIDLANDS|       39| #N/A|     #N/A|
|E12000005|   Region| 6085687|       WEST MIDLANDS|       48| #N/A|     #N/A|
|E12000006|   Region| 6468665|                EAST|       47| #N/A|     #N/A|
|E12000007|   Region| 8945309|              LONDON|       32| #N/A|     #N/A|
|E12000008|   Region| 9482507|          SOUTH EAST|       45| #N/A|     #N/A|
|E12000009|   Region| 5811259|          SOUTH WEST|       48| #N

In [8]:
import sys
import matplotlib # pip install matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatch

# Convert to Pandas
Pop = Cond_Pop.toPandas() # pip install setuptools
Inc = Cond_Inc.toPandas()
Sta = Statements.toPandas()
Dat = Cast_Data.toPandas()
Reg = Region.toPandas()

# Visualise data
Ages = Dat['All_ages'].values.tolist()
Tax = Dat['Tot_tax'].values.tolist()

# Create blank list
avg_tax = []

for i in range(len(Ages)):
    avg_tax.append(Ages[i]/Tax[i])

'''
# Create line of best fit
z = np.polyfit(x=Dat.loc[:,'All_ages'],y=Dat.loc[:,'Tot_tax'],deg=1)
p = np.poly1d(z)
Dat['Trendline'] = p(Dat.loc[:,'All_ages'])
'''
cat_var = []

# Create category
for i in range(len(avg_tax)):
    if avg_tax[i] > 469:
        cat_var.append('Lower')
    elif avg_tax[i] > 386 and avg_tax[i] < 469:
        cat_var.append('Lower middle')
    elif avg_tax[i] > 275 and avg_tax[i] < 386:
        cat_var.append('Upper middle')
    else:
        cat_var.append('Upper')

# Add cat_var to dataframe
Dat['Class'] = cat_var
Dat['Avg_tax'] = avg_tax

# Create the groups seen in graph
colours = {'Lower': 'red', 'Lower middle': 'pink', 'Upper middle': 'purple', 'Upper': 'blue'}
colour_list = [colours[group] for group in cat_var]

# Create scatter graph
legend_handles = [mpatch.Patch(color=colours['Lower'],label='Lower'),
                  mpatch.Patch(color=colours['Lower middle'],label='Lower middle'),
                  mpatch.Patch(color=colours['Upper middle'],label='Upper middle'),
                  mpatch.Patch(color=colours['Upper'],label='Upper')]

gro_ax = Dat.plot.scatter('Avg_tax','All_ages',c=colour_list,grid=True)
gro_ax.set_title('Tax vs. People')
gro_ax.set_xlabel('Tax')
gro_ax.set_ylabel('People')
gro_ax.legend(handles=legend_handles)

In [ ]:
# Creating target data
# Get region details into list
Geo = Dat['Geography'].values.tolist()
Code = Dat['Code'].values.tolist()

# Create blank 2d list
w, h = len(Geo), 2 
District = [[0]*h for i in range(w)]
c = ''

# Connect each district to its relative region
for i in range(0,len(Geo)):
    if Geo[i] == 'Region':
        c = Code[i]
    if Geo[i] == 'Country':
        c = 'Country'
    District[i][0] = c
    District[i][1] = Code[i]

# Get only codes in data dataframe
codes_dat = Dat['Code'].values.tolist()

# Add region to Data dataframe
reg = []
for i in range(0, len(District)):
    reg.append(District[i][0])

Dat['Region'] = reg

# Make a rating for each area
rating = []
Mor = Reg['Morrisons'].values.tolist()
Co_re = Reg['Code'].values.tolist()

# Go through each area, associate the number of shops to it, and make a rating
ratings = []
for area in range(0, len(avg_tax)):
    region = reg[area]
    if region == 'Country':
        ratings.append(0)
        continue
    for r in range(0, len(Mor)):
        if Co_re[r] == region:
            shops = Mor[r]
    rating = avg_tax[area] // int(shops)
    ratings.append(rating)

Dat['Rating'] = ratings
display(Dat)

All_ages  Num_ind   Mean  Median  Other_num_ind  Other_mean  Other_median  \
0    2711380      929  30200   24700            508        3840            31   
1    1153212      396  30500   25000            197        3800            29   
2    7600126     2690  32000   25000           1410        5930            43   
3    2948633     1030  31800   25000            500        5730            36   
4    1270162      440  30500   24200            265        6250            55   
5    1456562      503  30500   25000            235        4380            29   
6    5594125     1940  30900   24700           1130        5210            43   
7    1407072      478  29800   24400            260        4130            30   
8    2402161      816  31200   25200            472        5000            44   
9    4991265     1810  31700   25000           1010        5780            47   
10    811449      301  31500   24800            182        5640            44   
11    734015      282  33700   26700            178        6750            63   
12    782808      280  30200   24400            162        5350            49   
13    844494      310  32400   25100            186        5660            49   
14   6085687     2100  31600   25100           1160        5780            47   
15    898521      339  32500   25600            205        6340            51   
16    617823      242  37200   27500            152        7660            77   
17   2980936      943  30300   24600            475        4470            38   
18    614185      230  33100   25500            144        7260            66   
19   6468665     2410  38300   27700           1430        8340            77   
20    699573      281  40300   29500            167        7110            67   
21   1536118      563  40300   28500            341       11300            79   
22   1215387      492  46700   30600            311       10100           150   
23    931943      319  30500   25000            192        6120            77   
24    776442      272  32800   25100            176        6640            69   
25   8945309     3500  52700   32000           1730       12600           161   
26   9482507     3570  41900   28300           2250        8650            95   
27    555484      180  33400   24000            129        9540           134   
28   1428559      547  39800   28200            366        7440            71   
29   1610251      582  38800   26700            352        7680            74   
30    750230      293  42100   29800            185        9840            98   
31   1228671      490  56400   32600            339       11400           218   
32    900862      325  37700   27400            218        7760            82   
33   5811259     2080  32000   25100           1340        7030            74   
34    833408      269  30200   24200            199        6880           103   
35    659276      245  34400   25700            164        7580            65   
36   3164404     1090  29800   24800            625        4410            38   

    Tot_num_ind  Tot_mean  Tot_median  ...  Tax_median  Tot_tax  id  \
0          1230     31500       24900  ...        2230     5060   1   
1           508     31700       25200  ...        2290     2090   2   
2          3490     34000       25700  ...        2360    17200   3   
3          1290     33600       25500  ...        2350     6190   4   
4           600     32900       25200  ...        2260     2710   5   
5           642     32000       25200  ...        2310     2760   6   
6          2550     33000       25300  ...        2300    11700   7   
7           620     31300       24700  ...        2200     2510   8   
8          1060     32900       25500  ...        2330     4770   9   
9          2380     33800       25800  ...        2370    11500  10   
10          411     33200       25200  ...        2250     1910  11   
11          374     36500       27600  ...        2640     2080  12   
12

In [72]:
# Imports
#from sklearn.ensemble import RandomForestClassifier # pip3 install -U scikit-learn scipy matplotlib
# pip install setuptools
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error,mean_squared_error, r2_score

temp_dat = Dat
temp_dat = temp_dat.drop(columns=['Code','Area','Geography','Class','Region'])

# Select predictors
X=Dat[['Avg_tax']]
y=Dat['Rating']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create classifier
lr = LinearRegression()

# Create model
lr.fit(X_train, y_train)

# Print the regression coefficients
print ('Regression Coefficient (slope): ', lr.coef_)
print('Intercept: ', lr.intercept_)

# Predicts
y_pred = lr.predict(X_test)

# Evaluation
# Compute MAE, RMSE, and R2
mae = mean_absolute_error(y_test,y_pred)
rmse = np.sqrt(mean_squared_error(y_test,y_pred))
r2= r2_score(y_test,y_pred)

# print the evaluation metrics
print(f'Mean Absolute Error: {mae}')
print(f'Rppt Mean Squared Error: {rmse}')
print(f'Coefficient of determination: {r2}')


Regression Coefficient (slope):  [0.0195661]
Intercept:  0.02337490216634297
Mean Absolute Error: 1.1578123801256706
Rppt Mean Squared Error: 1.5087054335223626
Coefficient of determination: 0.6420729890689101


In [ ]:
# Visualise results